In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-05-29 11:10:39--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2025-05-29 11:10:39 (168 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Filter out implicit ratings
df_ratings_explicit = df_ratings[df_ratings['rating'] > 0]

# Merge ratings with book titles
df = pd.merge(df_ratings_explicit, df_books, on='isbn')

# Create pivot table (book-user matrix)
book_user_matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Convert to sparse matrix
book_user_sparse = csr_matrix(book_user_matrix.values)

# Fit NearestNeighbors model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)

# Store title index mapping for later lookup
book_titles = list(book_user_matrix.index)



<ipython-input-4-4f52615cee8f>:8: PerformanceWarning: The following operation may generate 9231436716 cells in the resulting pandas object.
  book_user_matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)


In [ ]:
# Function to return recommended books - this will be tested
def get_recommends(book=""):
    if book not in book_user_matrix.index:
        return [book, []]

    # Get the index of the book
    book_idx = book_user_matrix.index.get_loc(book)

    # Find 6 neighbors (including the book itself at position 0)
    distances, indices = model.kneighbors(book_user_matrix.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6)

    recommended_books = []
    for i in range(1, len(indices[0])):  # Skip the input book itself
        title = book_user_matrix.index[indices[0][i]]
        dist = distances[0][i]
        recommended_books.append([title, dist])

    return [book, recommended_books]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()